# Hidden Markov Models, Vitrebi Algorithm

# Problem Statement:

## Modify the Viterbi algorithm to solve the problem of unknown words using at least two techniques. 


1. Which tag class do you think most unknown words belong to? Can you identify rules (e.g. based on morphological cues) that can be used to tag unknown words? You may define separate python functions to exploit these rules so that they work in tandem with the original Viterbi algorithm.
2. Why does the Viterbi algorithm choose a random tag on encountering an unknown word? Can you modify the Viterbi algorithm so that it considers only one of the transition or emission probabilities for unknown words?

In [ ]:
#importing necessary libraries
import numpy as np
import pandas as pd
import random
import re
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
from sklearn.model_selection import train_test_split

In [ ]:
#Importing NLTK libraries and resources
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')
from nltk.tokenize import word_tokenize

In [ ]:
# reading the Treebank tagged with universal tagset
universalTreebankDataset = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [ ]:
pp=pprint.PrettyPrinter()

In [ ]:
# looking at the tags and first few sentences
pp.pprint(universalTreebankDataset[:40])

In [ ]:
#test-train split: 95-5
train_set, validation_set = train_test_split(universalTreebankDataset,test_size=0.05,random_state=1331,shuffle=True)

In [ ]:
print(len(train_set))
print(len(validation_set))

In [ ]:
pp.pprint(train_set[:20])
pp.pprint(validation_set[:10])

In [ ]:
#obtaining the nuber of tuples which are tagged in train set:
taggedWordsInTrainDataset=[t for s in train_set for t in s ]
print(len(taggedWordsInTrainDataset))
print(taggedWordsInTrainDataset)


In [ ]:
#finding teh tagged words i.e. knows words:
knownWords=[taggedWordSet[0] for taggedWordSet in taggedWordsInTrainDataset ]
print(len(set(knownWords)))

In [ ]:
#finding the number of tokens:
tagsUsed=[taggedWordSet[1] for taggedWordSet in taggedWordsInTrainDataset ]
print(len(set(tagsUsed)))
print(set(tagsUsed))

In [ ]:
# Reading the test data provided:
file=open('Sample_Test_Sentences.txt','r')
testData=file.readlines();
file.close();
print(testData)


# Vanilla HMM with Vitrebi Algorithm

1. HMM- Hidden Markov Models use stocastic methods to model sequential processes.
2. Using the first order markovian assumption, the probability of a state depends only on the probability of the previous state.
3. HMM have hidden states, which emit the probabilty of observations. 
4. These probabilties are called the Emission probabilites.
5. The probability of one state coming after another is called transition probabitliy, as there is a tarnsition from one state to another.

Using HMM, we will assign a tag (say t) to a word (say w), such that the liklihood P(t|w) is maximum.
Now, 
$P(t|w) = {P(w|t)*P(t)}/P(w)$
is the formula for the liklihood.

To compute it we can use the count of the tags and words computed above.

## Emission Probabilties

In [ ]:
def EmissionProbability_WordgivenTag(word, tag, train_data = taggedWordsInTrainDataset):
    tag_list = [pair for pair in train_data if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

## Transition Probability

In [ ]:
def TransitionProbability_t2_given_t1(t2, t1, train_bag = taggedWordsInTrainDataset):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

Now lets create a transion matrix that will give the probability of state transiton from one tag(ti) to another(tj).

In [ ]:
tagTransiontMatrix = np.zeros((len(set(tagsUsed)), len(set(tagsUsed))), dtype='float32')
for i, t1 in enumerate(list(set(tagsUsed))):
    for j, t2 in enumerate(list(set(tagsUsed))): 
        tagTransiontMatrix[i, j] = TransitionProbability_t2_given_t1(t2, t1)[0]/TransitionProbability_t2_given_t1(t2, t1)[1]

In [ ]:
tagTransiontMatrix

In [ ]:
tagTransionDF=pd.DataFrame(tagTransiontMatrix, columns = list(set(tagsUsed)), index=list(set(tagsUsed)))
display(tagTransionDF)

In [ ]:
plt.figure(figsize=(18, 12))
sns.heatmap(tagTransionDF)
plt.show()


# Vitrebi Algorithm

In [ ]:
# Viterbi Heuristic
class Viterbi:
    this.taggedWordsInTrainDataset=""
    
    def tag(words, train_bag = taggedWordsInTrainDataset):
        state = []
        T = list(set([pair[1] for pair in train_bag]))

        for key, word in enumerate(words):
            #initialise list of probability column for a given observation
            p = [] 
            for tag in T:
                if key == 0:
                    transition_p = tagTransionDF.loc['.', tag]
                else:
                    transition_p = tagTransionDF.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = EmissionProbability_WordgivenTag(words[key], tag)[0]/EmissionProbability_WordgivenTag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)

            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
        return list(zip(words, state))
    def EmissionProbability_WordgivenTag(word, tag, train_data = taggedWordsInTrainDataset):
        tag_list = [pair for pair in train_data if pair[1]==tag]
        count_tag = len(tag_list)
        w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
        count_w_given_tag = len(w_given_tag_list)

        return (count_w_given_tag, count_tag)
    def TransitionProbability_t2_given_t1(t2, t1, train_bag = taggedWordsInTrainDataset):
        tags = [pair[1] for pair in train_bag]
        count_t1 = len([t for t in tags if t==t1])
        count_t2_t1 = 0
        for index in range(len(tags)-1):
            if tags[index]==t1 and tags[index+1] == t2:
                count_t2_t1 += 1
        return (count_t2_t1, count_t1)

In [ ]:
# list of sents
test_run = validation_set
test_run_base = [tup for sent in test_run for tup in sent]
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
print(test_run)

In [ ]:
print(test_run_base)

In [ ]:
print(test_tagged_words)

In [ ]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

### Tagging Accuracy

In [ ]:
# accuracy
def FindAccuracy(tagged_seq,test_run_base):
    check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
    accuracy = len(check)/len(tagged_seq)
    print("Validation Accuracy= "+str(accuracy));
    print("time taken= "+str(difference))
    
FindAccuracy(tagged_seq,test_run_base)

## Incorrect tagged cases:

In [ ]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

## Running the model with Test Data

In [ ]:
for testSentence in testData:
    words = word_tokenize(testSentence)
    tagged_seq = Viterbi(words)
    difference = end-start
    print(tagged_seq)
    print(pos_tag(word_tokenize(testSentence),tagset='universal'))

In [ ]:
FindAccuracy(tagged_seq,test_run_base)

### Finding the unknown words

In [ ]:
from nltk.tag import StanfordNERTagger
from nltk import pos_tag
for
pos_tag(word_tokenize(testData[1]),tagset='universal')

# Modifications to Address the issues with unknown words

## METHOD 1

### Tagging Accuracy

## METHOD 2

### Tagging Accuracy

## COMPARISION: Tagging Accuracy

# VANILLA TAGGER VS MODIFIED TAGGER: TEST CASES  